## Инкапсуляция


В программировании, под словом "инкапсуляция" могут понимать два различных понятия:

1. Механизм языка программирования по ограничению доступы к определеным компонентам объекта (иногда это называют **сокрытием**)
2. Возможность языка программирования "упаковывать" данные с методами, предназначенными для обработки этих данных

Давайте рассмотрим как в Python реализовано ограничение доступа к аттрибутам объекта.

In [2]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [3]:
import random
import string

symbols = list(string.printable)
random.shuffle(symbols)

class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        k = random.randint(7, 15)
        self.password = ''.join(random.choices(symbols, k=k))

In [6]:
ivan = Person('Ivan', 'Ivanov')

In [7]:
ivan.password

'r}Qa[z\\\\UM\tIN_'

Запрет на уровне соглашения

In [8]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        k = random.randint(7, 15)
        self._password = ''.join(random.choices(symbols, k=k))

In [9]:
ivan = Person('Ivan', 'Ivanov')

In [10]:
ivan._password

'f/d^\x0cdPu'

In [11]:
ivan._password = 'Я поменялся!'

In [12]:
ivan._password

'Я поменялся!'

Аттрибут по прежнему доступен для просмотра и изменения:

In [13]:
ivan._password = 'Я поменял паспорт'
ivan._password

'Я поменял паспорт'

Возможно ваша IDE даст вам понять о том что такие аттрибуты трогать нельзя:

![image](https://i.imgur.com/tbglQYt.png)

Чуть более строгий запрет

In [14]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        k = random.randint(7, 15)
        self.__password = ''.join(random.choices(symbols, k=k))

In [15]:
ivan = Person('Ivan', 'Ivanov')

In [16]:
ivan.__password

AttributeError: 'Person' object has no attribute '__password'

Но это можно обойти:

In [17]:
ivan._Person__password

"vc<N)rEK'@sZiR^"

In [18]:
ivan._Person__password = 15

In [19]:
ivan._Person__password

15

Однако

In [20]:
ivan.__password = 'Не понял...'

In [21]:
dir(ivan)

['_Person__password',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__password',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'name',
 'surname']

Т.е. происходит переименовывание нашего аттрибута. Важно что внутри класса он будет доступен по прежнему имени.

Поскольку метод - частный случай аттрибута, то все вышенаписаное относится и к методам. 

## Наследование. Super.

Под наследованием понимается возможность создать новые классы на основе уже существующих. Класс от которого наследуется текущий называют родительским.

Рассмотрим пример:

In [22]:
# Создадим класс на основе класса list, добавив два новых метода:
# 1. is_even - четна ли длина списка
# 2. square - возвращаем список все элементы которого квадрат данного

class MyList(list):
    def is_even(self):
        return len(self) % 2 == 0
    def square(self):
        return [x**2 for x in self]

In [26]:
ls = MyList()
print(ls)

ls.extend([1,2,3,4,5])
print(ls)
print(ls.is_even())

ls.append(10)
print(ls)
print(ls.is_even())


print(ls.square())

[]
[1, 2, 3, 4, 5]
False
[1, 2, 3, 4, 5, 10]
True
[1, 4, 9, 16, 25, 100]


Таким образом, если аттрибут не найден в текущем классе, то Python идет искать в класс родитель, это касается и конструктора.

 - ```isinstance(x, A)```: является ли x экземпляром A (на 2-ом месте может стоять кортеж, тогда достаточно того чтобы x был экземпляром одного из элементов кортежа)
 - ```issubclass(A, B)```: является ли A потомком B (на 2-ом месте может стоять кортеж, тогда достаточно того чтобы A был потомком одного из элементов кортежа)
 - ```cls.mro()``` - пока что посмотреть на цепочку наследований (при множественном наследовании - список в порядоке поиска метода для данного объекта)

In [32]:
isinstance(ls, list)

True

In [37]:
issubclass(MyList, MyList)

True

In [39]:
MyList.mro()

[__main__.MyList, list, object]

In [43]:
# Что происходит?

class A:
    pass

a = A()

In [44]:
A.mro()

[__main__.A, object]

In [46]:
issubclass(A, object)

True

In [49]:
isinstance(a, object)

True

Еще один пример

In [51]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        Person.__init__(self, name, surname)
        self.courses = args

In [52]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [53]:
print(s.name, s.surname)

Ivan Sidorov


Функция **super** позволяет наследовать аттрибуты родителя.

In [54]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        super(Student, self).__init__(name, surname)
        # Person.__init__(self, name, surname)
        self.courses = args

In [55]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [56]:
print(s.name, s.surname)

Ivan Sidorov


In [57]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        
class Student(Person):
    def __init__(self, name, surname, *args):
        super().__init__(name, surname)
        self.courses = args

In [58]:
s = Student('Ivan', 'Sidorov', 'Math', 'Physics', 'Russian', 'History')
s.courses

('Math', 'Physics', 'Russian', 'History')

In [59]:
print(s.name, s.surname)

Ivan Sidorov


Упражнение:

Создайте класс прямоугольника, который

1. Инициализирует стороны
2. Имеет метод для подсчета площади
3. Имеет метод для подсчета периметра

Затем, создайте класс квадрата, наследующийся от прямоугольника, таким образом чтобы все методы родителя
работали корректно для квадрата.

In [60]:
class Rectangle:
    def __init__(self, a, b):
        self.a = a
        self.b = b
    def area(self):
        res = self.a * self.b
        print(f'Our area is {res}')
        return res
    def perimeter(self):
        res = 2*(self.a + self.b)
        print(f'Our perimeter is {res}')
        return res

In [74]:
class Square(Rectangle):
    def __init__(self, a):
        super().__init__(a, a)

In [75]:
s = Square(10)
s.area()

Our area is 100


100

In [76]:
s.perimeter()

Our perimeter is 40


40

Важно: object - предок для всех классов, базовый класс

In [77]:
def f():
    pass

type(f).mro()

[function, object]

## Полиморфизм

Полиморфизм - единый интерфейс для сущностей разных типов.

Например, для оператора ```+```

In [78]:
print(1 + 2)
print('ab' + 'cd')

3
abcd


In [79]:
{1, 2, 3} & {3, 4, 5}

{3}

In [80]:
10 & 2

2

Для функций:

In [81]:
print(len([1,2,3]))
print(len({1,2,3}))

3
3


In [82]:
print(len({1: 'hi', 2:'bye'}))

2


Для классов:

In [84]:
class Cat:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a cat. My name is {self.name}. I am {self.age} years old.")

    def make_sound(self):
        print("Meow")

class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a dog. My name is {self.name}. I am {self.age} years old.")

    def make_sound(self):
        print("Bark")

In [85]:
cat1 = Cat("Kitty", 2.5)
dog1 = Dog("Fluffy", 4)

for animal in (cat1, dog1):
    animal.make_sound()
    animal.info()
    animal.make_sound()

Meow
I am a cat. My name is Kitty. I am 2.5 years old.
Meow
Bark
I am a dog. My name is Fluffy. I am 4 years old.
Bark


Для классов при наследовании:

In [87]:
class Animal:
    def __init__(self, name, legs, scariness):
        self.name = name 
        self.legs = legs
        self.scariness = scariness
    
    def introduce(self): 
        print("Hello! My name is %s!" % self.name)
    
    def sound(self):
        print("Sound!")

class Mammal(Animal):
    
    def __init__(self, name, scariness): 
        super().__init__(name=name, legs=4, scariness=scariness)

In [88]:
class Cat(Mammal):
    
    # переопределяем метод sound, чтобы кошка мяукала
    def sound(self): 
        print("Meow!")
    
class Dog(Mammal):
    
    # переопределяем метод sound, чтобы собака гавкала
    def sound(self): 
        print("Woof!")

class Cow(Mammal):
    
    # переопределяем метод sound, чтобы корова мычала
    def sound(self):
        print("Mooo!")

In [89]:
cat = Cat(name='Cat', scariness=2)
dog = Dog(name='Dog', scariness=3)
cow = Cow(name='Cow', scariness=1)

Одинаковый интерфейс (название функциии и аргументы), разные действия в зависимости от конкретного класса-потомка:


In [90]:
for animal in [cat, dog, cow]:
    print("%s wants to say something..." % animal.name)
    animal.sound()

Cat wants to say something...
Meow!
Dog wants to say something...
Woof!
Cow wants to say something...
Mooo!


## Множественное наследование. MRO.

Python поддерживает множественное наследование.

Пример:


In [61]:
class B:
    att_b = 'I am the attribute of B!'

class C:
    att_c = 'I am the attribute of C!'

class A(B, C):
    pass

In [62]:
a = A()
print(a.att_b, a.att_c, sep='\n')

I am the attribute of B!
I am the attribute of C!


Упражнение: Adress, Person - соединить в один с помощью множественного наследования.

In [138]:
class Address:
    def __init__(self, street, city):
        self.street = str(street)
        self.city = str(city)
        
    def show(self):
        print(self.street)
        print(self.city)

class Person:
    def __init__(self, name, email):
        self.name = name
        self.email= email
        
    def show(self):
        print(self.name + ' ' + self.email)

In [139]:
class Person2(Person, Address):
    def __init__(self, street, city, name, email):
        Person.__init__(self, name, email)
        Address.__init__(self, street, city)

Возможность создавать множественные наследования может вести к довольно громоздким и запутанным графам отношений между классами. В таком случае, если у текущего класса есть несколько предков, у которых аттрибуты имеют одинаковые имена, непонятно в каком порядке этот аттрибут ищется.

Например:

![ex](https://i.imgur.com/Eemp2jY.png)

In [135]:
class O:
    pass
class A(O):
    pass
class B(O):
    pass
class C(O):
    pass
class D(O):
    pass
class E(O):
    pass

class K1(A, B, C):
    pass
class K2(B, D):
    pass
class K3(C, D, E):
    pass

class Z(K1, K2, K3):
    pass

In [136]:
def print_mro(T):
    print(*[c.__name__ for c in T.mro()], sep=' -> ')
print_mro(Z)

Z -> K1 -> A -> K2 -> B -> K3 -> C -> D -> E -> O -> object


В каком порядке?

Ответ: алгоритм **C3 linearization**. О нем можно почитать здесь:

- https://en.wikipedia.org/wiki/C3_linearization
- https://dl.acm.org/doi/pdf/10.1145/236337.236343

Важно понимать что сохраняется здравый смысл:
- если имеем ```class A(B, C, D)```, то в MRO однозначно имеем что A идет до В, В до C и С до D (возможно между ними что-то есть)
- родители идут до grandparents и т.д.
- object всегда последний элемент

Поэтому в простых случаях, типо diamond inheritance все ясно.

In [ ]:
class X:
    pass
class Y:
    pass
class A(X, Y):
    pass
class B(Y, X):
    pass
class G(A, B):
    pass

# Что произошло? Придумайте еще похожий пример

## Итераторы

Различают два понятия: **iterator** и **iterable**

**iterable** - объект по которому можно итерироваться. 

Примеры iterable:
- list
- tuple
- string
- dict
- set
- файл
- любой кастомный класс, в котором реализован метод ```__iter__```
- любой кастомный класс, в котором реализован метод ```__getitem__```

К объектам iterable можно применять циклы ```for``` и методы типа ```zip```, ```map``` в которых нужны последовательности элементов.

**iterator** - объект у которого определен метод ```__next__```.

In [91]:
ls = [1, 2, 3, 4, 5]
for x in ls:
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

Цикл for "под капотом"

In [92]:
ls = [1, 2, 3, 4, 5]
iterator = ls.__iter__()
while True:
    try:
        x = iterator.__next__()
    except StopIteration:
        break
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

В Python есть встроенные функции ```iter``` и ```next```, котоорые
эквивалентны ```__iter__``` и ```__next__```

In [94]:
ls = [1, 2, 3, 4, 5]
iterator = iter(ls)
while True:
    try:
        x = next(iterator)
    except StopIteration:
        break
    print(x, end=' ')
    # тело цикла

1 2 3 4 5 

В Python есть встроенные функции ```next``` и ```iter```, которые являются аналогами одноименных методов.

Давайте напишем собственный итератор, который ведет себя аналогично итератору для ```range```:

In [104]:
class MyRange:
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        return self
    def __next__(self):
        if self.start < self.end:
            ret = self.start
            self.start += self.step
            return ret
        else:
            raise StopIteration

In [105]:
r = MyRange(1, 10, 2)


In [107]:
next(r)

1

In [108]:
r.start

3

In [100]:
r = MyRange(1, 10, 2)
for x in r:
    print(x)

1
3
5
7
9


Тоже самое, но итератор отдельно от объекта:

In [101]:
class MyIterator:
    def __init__(self, obj):
        self.start = obj.start
        self.end = obj.end
        self.step = obj.step
    def __next__(self):
        if self.start < self.end:
            ret = self.start
            self.start += self.step
            return ret
        else:
            raise StopIteration
        

class MyRange:
    def __init__(self, start, end, step=1):
        self.start = start
        self.end = end
        self.step = step
    def __iter__(self):
        return MyIterator(self)

In [103]:
r = MyRange(1, 10, 2)
for x in r:
    print(x, end=' ')

1 3 5 7 9 

In [ ]:
r.start

Теперь мы умеем прикрупчивать к своим классам возможность перечисления!

## Генераторы

Давайте поговорим про оператор ```yeild```. Это почти тоже самое что ```return```, но есть существенное различее: когда интепретатор доходит до ```yeild``` он возвращает то на что указывает ```yeild``` и затем **останавливает** исполнение функции. Если позвать функцию заново - мы продолжим с момента на котором остановились:

In [141]:
import time

In [166]:
def Hello():
    yield f'current time is {time.strftime("%H:%M:%S %d.%m.%Y")}'
    yield f'current time is {time.strftime("%H:%M:%S %d.%m.%Y")}'    

In [167]:
generator = Hello()
generator

<generator object Hello at 0x7f97d7e267d8>

In [168]:
next(generator)

'current time is 17:32:57 23.11.2022'

In [169]:
next(generator)

'current time is 17:33:00 23.11.2022'

In [170]:
next(generator)

StopIteration: 

Т.е. генераторы являются итераторами:

In [171]:
'__iter__' in dir(generator)

True

In [173]:
'__next__' in dir(generator)

True

In [175]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            result = i**2
            i += 1
            yield result
        else:
            return 'all done!'

In [181]:
sq = squares(3)

In [182]:
for x in sq:
    print(x)

0
1
4


Напишем генератор для range

In [183]:
def my_range(start, end, step):
    while start < end:
        yield start
        start += step

In [184]:
r = my_range(1, 10, 2)

In [185]:
for x in r:
    print(x)

1
3
5
7
9


In [186]:
list(my_range(1, 10, 2))

[1, 3, 5, 7, 9]